In [ ]:
from sklearn.preprocessing import normalize
from sklearn.preprocessing import scale
from sklearn.cross_validation import cross_val_score
from hyperopt import fmin, tpe, hp, STATUS_OK,Trials
from sklearn.cross_validation import cross_val_score, KFold

from sklearn.preprocessing import normalize
from sklearn.preprocessing import scale
from sklearn.cross_validation import cross_val_score
from hyperopt import fmin, tpe, hp, STATUS_OK,Trials
from sklearn.cross_validation import cross_val_score, KFold

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import seaborn as sns
sns.set_style("darkgrid", {"axes.facecolor": "0.9"})
sns.set_context("poster")

from sklearn.preprocessing import normalize, MinMaxScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import validation_curve

In [ ]:
def go_back(look_back, df):
    #Create previous Day price column 
    
    for i in range(look_back):
        name = 'Previous_Day_Price' + str(i)
        df[name] = df['Weighted_Price'].shift(+1+i)
        df.dropna(how='any', inplace=True)
        
    return df

In [ ]:
df = pd.read_csv('tweet_n_btc_day.csv')
#Drop np.nan
df.dropna(how='any', inplace=True)

df = df[['Weighted_Price']]

#Go back
look_back = 5
df = go_back(look_back, df)

df = df.values

# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
df = scaler.fit_transform(df)

dataset = np.delete(df,(0), axis=1)
labels = np.delete(df,(1,2,3,4,5), axis=1)

x_train, x_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.4)

In [ ]:
def hyperopt_train_test(params):

    clf =  GradientBoostingRegressor(**params)
    clf.fit(x_train, y_train)
    
    prediction = clf.predict(x_train)
    print (math.sqrt(mean_squared_error(prediction,y_train)) )
   
    return math.sqrt(mean_squared_error(prediction,y_train))

In [ ]:
best = 0
def f(params):
    global best
    acc = hyperopt_train_test(params)
    if acc > best:
        best = acc
        print ('new best:', best, params)
    return {'loss': acc, 'status': STATUS_OK}

In [ ]:
GradientBoosting4space = {

    'max_depth': hp.quniform('max_depth',1,20,1), 
    'max_depth': hp.quniform('max_depth',1,20,1),
    'min_weight_fraction_leaf': hp.uniform('min_weight_fraction_leaf', 0, 0.5),
}

In [ ]:
trials = Trials()
best = fmin(f, GradientBoosting4space , algo=tpe.suggest, max_evals=1000, trials=trials)
print ('best:')
print (best)